In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# Main

In [2]:
! pip install Cython==0.29.36
! pip install fasttext==0.8.3
! pip install kenlm
! pip install torchtext==0.4.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 29.0 MB/s eta 0:00:00
  Attempting uninstall: Cython
    Found existing installation: Cython 3.0.5
    Uninstalling Cython-3.0.5:
      Successfully uninstalled Cython-3.0.5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.0/74.0 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fasttext: filename=fasttext-0.8.3-cp310-cp310-linux_x86_64.whl size=1481037 sha256=eddfd5cdcdac0c344df3614f12e10596aaef09143c67cb9000b119c422e458df
  Stored in directory: /root/.cache/pip/wheels/2b/85/8b/3037ff2b0c1297bd87e6deee5c9a06e4b4da5f4c448c66f5f5
Successfully built fasttext
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.4/427.4 kB 5.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for kenlm: filename=kenlm-0.2.0-cp310-cp310-linux_x86_64.whl size=3184428 sha256=059ae35314b85268bc88129d2c4b601

In [8]:
%cd /content/gdrive/Shareddrives/cici 544/style-transformer

/content/gdrive/Shareddrives/cici 544/style-transformer


In [4]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import torch
import time
from data import load_dataset
from models import StyleTransformer, Discriminator
from train import train, auto_eval
from torchsummary import summary

class Config():
    data_path = './data/amazon35/'
    log_dir = 'runs/exp'
    dataName = 'amazon35'
    save_path = './save'
    pretrained_embed_path = './embedding/'
    device = torch.device('cuda' if True and torch.cuda.is_available() else 'cpu')
    discriminator_method = 'Cond' # 'Multi' or 'Cond'
    load_pretrained_embed = False
    min_freq = 3
    max_length = 36
    embed_size = 200
    d_model = 200
    h = 4
    num_styles = 2
    num_classes = num_styles + 1 if discriminator_method == 'Multi' else 2
    num_layers = 4
    batch_size = 64
    lr_F = 0.001
    lr_D = 0.001
    L2 = 0
    iter_D = 10
    iter_F = 5
    F_pretrain_iter = 500
    log_steps = 5
    eval_steps = 25
    learned_pos_embed = True
    dropout = 0
    drop_rate_config = [(1, 0)]
    temperature_config = [(1, 0)]

    slf_factor = 0.25
    cyc_factor = 0.5
    adv_factor = 1

    inp_shuffle_len = 0
    inp_unk_drop_fac = 0
    inp_rand_drop_fac = 0
    inp_drop_prob = 0

def main():
    config = Config()
    train_iters, dev_iters, test_iters, vocab = load_dataset(config)
    print('Vocab size:', len(vocab))
    model_F = StyleTransformer(config, vocab).to(config.device)
    model_D = Discriminator(config, vocab).to(config.device)
    # testmodel_F = StyleTransformer(config, vocab)

    # pretrained_F_dict = torch.load('/content/gdrive/MyDrive/CSCI 544/project/style-transformer/yelp_glove200_F.pth', map_location=torch.device('cpu'))
    # pretrained_D_dict = torch.load('/content/gdrive/MyDrive/CSCI 544/project/style-transformer/yelp_glove200_D.pth', map_location=torch.device('cpu'))

    # #  load generator - yelp pretrain model
    # model_F_dict = model_F.state_dict()
    # pretrained_F_dict = {k: v for k, v in pretrained_F_dict.items() if 'generator' not in k and 'embed' not in k}
    # model_F_dict.update(pretrained_F_dict)
    # model_F.load_state_dict(model_F_dict)

    # model_F.encoder.requires_grad_(False)
    # model_F.decoder.requires_grad_(False)
    # model_F.decoder.generator.requires_grad_(True)

    # model_F.to(config.device)

    # model_D_dict = model_D.state_dict()
    # pretrained_D_dict = {k: v for k, v in pretrained_D_dict.items() if k.startswith('embed') == False}
    # model_D_dict.update(pretrained_D_dict)
    # model_D.load_state_dict(model_D_dict)

    # model_D.encoder.requires_grad_(False)
    # model_D.to(config.device)

    # # for name, param in model_F.named_parameters() :
    # #   print(name, param.requires_grad)

    # print(config.discriminator_method)

    train(config, vocab, model_F, model_D, train_iters, dev_iters, test_iters)


if __name__ == '__main__':
    main()

Vocab size: 21760
Save Path: ./save/Nov28182818
Model F pretraining......
[iter: 10] slf_loss:41.9802, rec_loss:0.0000
[iter: 20] slf_loss:32.8043, rec_loss:0.0000
[iter: 30] slf_loss:29.4170, rec_loss:0.0000
[iter: 40] slf_loss:28.6955, rec_loss:0.0000
[iter: 50] slf_loss:27.8585, rec_loss:0.0000
[iter: 60] slf_loss:27.0219, rec_loss:0.0000
[iter: 70] slf_loss:27.5704, rec_loss:0.0000
[iter: 80] slf_loss:26.9756, rec_loss:0.0000
[iter: 90] slf_loss:26.6422, rec_loss:0.0000
[iter: 100] slf_loss:26.2432, rec_loss:0.0000
[iter: 110] slf_loss:25.8018, rec_loss:0.0000
[iter: 120] slf_loss:26.6324, rec_loss:0.0000
[iter: 130] slf_loss:26.2125, rec_loss:0.0000
[iter: 140] slf_loss:25.7834, rec_loss:0.0000
[iter: 150] slf_loss:25.5477, rec_loss:0.0000
[iter: 160] slf_loss:24.6829, rec_loss:0.0000
[iter: 170] slf_loss:25.1800, rec_loss:0.0000
[iter: 180] slf_loss:24.5312, rec_loss:0.0000
[iter: 190] slf_loss:24.3089, rec_loss:0.0000
[iter: 200] slf_loss:23.8839, rec_loss:0.0000
[iter: 210] slf

/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

******************** neg sample ********************
[gold] item arrived with scratches and dings on the base plate .
[raw ] item arrived with papers and far on the base model .
[rev ] item arrived with papers and far on the base model .
[ref ] i should have returned these while i could they are waxy and light .

******************** neg sample ********************
[gold] the black seems to work ok but the color ones either run out too quickly or my printer is having trouble figuring out they are there .
[raw ] the black seems to work ok but the color ones either run out too quickly or my printer is having trouble out they are there .
[rev ] the black seems to work ok but the color ones either run out too quickly or my printer is having trouble out they are there .
[ref ] too small

******************** neg sample ********************
[gold] it did not work at all it did reproduce images that represented the slide but they absolutely very low quality <pad>
[raw ] it did not work at all

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-6-f7290b74b7b0>", line 88, in <cell line: 87>
    main()
  File "<ipython-input-6-f7290b74b7b0>", line 84, in main
    train(config, vocab, model_F, model_D, train_iters, dev_iters, test_iters)
  File "/content/gdrive/MyDrive/CSCI 544/project/style-transformer/train.py", line 323, in train
    torch.save(model_F.state_dict(), config.save_folder + '/ckpts/' + str(global_step) + '_F.pth')
  File "/usr/local/lib/python3.10/dist-packages/torch/serialization.py", line 618, in save
    with _open_zipfile_writer(f) as opened_zipfile:
  File "/usr/local/lib/python3.10/dist-packages/torch/serialization.py", line 492, in _open_zipfile_writer
    return container(name_or_buffer)
  File "/usr/local/lib/python3.10/dist-packages/torch/serialization.py", line 463, in __init__
    super(

# **Retrained hw1 data**

In [ ]:
! pip install contractions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 15.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
import re

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
train_list = []
train_number = 0
test_number = 0
with open('/content/gdrive/MyDrive/CSCI 544/project/style-transformer/data/amazon35/train.pos', 'r') as file :
    text = file.read()
    for t in text.split("\n"):
      if len(t.split()) > 0 :
        train_list.append([t, 1])
        train_number += 1

with open('/content/gdrive/MyDrive/CSCI 544/project/style-transformer/data/amazon35/train.neg', 'r') as file :
    text = file.read()
    for t in text.split("\n"):
      if len(t.split()) > 0 :
        train_list.append([t, 0])
        train_number += 1

with open('/content/gdrive/MyDrive/CSCI 544/project/style-transformer/data/amazon35/test.pos', 'r') as file :
    text = file.read()
    for t in text.split("\n"):
      if len(t.split()) > 0 :
        train_list.append([t, 1])
        test_number += 1

with open('/content/gdrive/MyDrive/CSCI 544/project/style-transformer/data/amazon35/test.neg', 'r') as file :
    text = file.read()
    for t in text.split("\n"):
      if len(t.split()) > 0 :
        train_list.append([t, 0])
        test_number += 1

In [ ]:
print("train number", train_number, " test number", test_number)

train number 504976  test number 1000


In [ ]:
df = pd.DataFrame(train_list, columns = ['texts', 'rate'])
df.head()

,texts,rate
0,was a little crooked when i received it but it...,1
1,this is a great way to freshen up a workstatio...,1
2,quality product,1
3,finally hit pay dirt . after searching i final...,1
4,excellent quality and exactly what i was needi...,1


In [ ]:
df['texts'] = [re.sub(r'[^a-zA-Z]',' ',str(x)) for x in df['texts']]

In [ ]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

In [ ]:
df['texts'] = df['texts'].apply(lambda x : [word for word in x.split(' ') if word not in stop_words])
df['texts'] = df['texts'].apply(lambda x : [word for word in x if word != ''])

In [ ]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
lemmantizer = WordNetLemmatizer()

In [ ]:
def get_wordnet_pos(word) :
  tag = nltk.pos_tag([word])[0][1][0].upper()
  tag_dict = {"J": wordnet.ADJ,
              "N": wordnet.NOUN,
              "V": wordnet.VERB,
              "R": wordnet.ADV}
  return tag_dict.get(tag, wordnet.NOUN)

df['texts'] = df['texts'].apply(lambda x : [lemmantizer.lemmatize(word,get_wordnet_pos(word)) for word in x])

In [ ]:
df['texts'] = df['texts'].apply(lambda x : " ".join(x))
text_data = [''.join(tokens).strip() for tokens in df['texts']]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

bow = CountVectorizer(max_features=700)
bow_matrix = bow.fit_transform(text_data).toarray()

bow_df = pd.DataFrame(bow_matrix, columns=bow.get_feature_names_out())

In [ ]:
from sklearn import svm
from sklearn.metrics import precision_recall_fscore_support

train_x = bow_df[:504976]
train_y = df.loc[:504975,['rate']]
test_x = bow_df[504976:]
test_y = df.loc[504976:, ['rate']]

print(len(train_x), ' ', len(train_y))
clf = svm.LinearSVC(max_iter=3000)
clf.fit(train_x, train_y)

y_pred = clf.predict(test_x)
precision_recall_fscore_support(test_y, y_pred, average='macro')

504976   504976


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


(0.8373819163292847, 0.8360000000000001, 0.8358318918572618, None)

# **Discriminator**
First, we train the model in the paper using amazon dataset35. And then we use this pretrained discriminator to do the comparison with hw1


In [26]:
from train import get_lengths, batch_preprocess
def predict(config, vocab, model_D, test_iters) :
  model_D.eval()

  batch_iters = iter(test_iters)
  prediction = []
  labels = []

  for i in range(0, 8) :
    batch = next(batch_iters)

    pad_idx = vocab.stoi['<pad>']
    eos_idx = vocab.stoi['<eos>']
    vocab_size = len(vocab)
    inp_tokens, inp_lengths, raw_styles = batch_preprocess(batch, pad_idx, eos_idx)
    # print(inp_tokens.size())
    rev_styles = 1 - raw_styles

    raw_gold_log_probs = model_D(inp_tokens, inp_lengths, torch.ones_like(raw_styles))
    pred = torch.argmax(raw_gold_log_probs, dim=1)
    # print(inp_tokens)
    # print(raw_styles)
    pred = pred.tolist()

    prediction = prediction + pred
    labels = labels + raw_styles.tolist()

  print(precision_recall_fscore_support(prediction, labels, average='macro'))

In [11]:
import torch
import time
from data import load_dataset
from models import StyleTransformer, Discriminator
from train import train, auto_eval
from torchsummary import summary

class Config():
    data_path = './data/amazon35/'
    log_dir = 'runs/exp'
    dataName = 'amazon35'
    pretrained_embed_path = './embedding/'
    device = torch.device('cuda' if True and torch.cuda.is_available() else 'cpu')
    discriminator_method = 'Cond' # 'Multi' or 'Cond'
    load_pretrained_embed = True
    min_freq = 3
    max_length = 36
    embed_size = 200
    d_model = 200
    h = 4
    num_styles = 2
    num_classes = num_styles + 1 if discriminator_method == 'Multi' else 2
    num_layers = 4
    batch_size = 64
    lr_F = 0.0001
    lr_D = 0.0001
    L2 = 0
    iter_D = 10
    iter_F = 5
    F_pretrain_iter = 800
    log_steps = 5
    eval_steps = 25
    learned_pos_embed = True
    dropout = 0
    drop_rate_config = [(1, 0)]
    temperature_config = [(1, 0)]

    slf_factor = 0.25
    cyc_factor = 0.5
    adv_factor = 1

    inp_shuffle_len = 0
    inp_unk_drop_fac = 0
    inp_rand_drop_fac = 0
    inp_drop_prob = 0

config = Config()
train_iters, dev_iters, test_iters, vocab = load_dataset(config)
print('Vocab size:', len(vocab))

model_D = Discriminator(config, vocab)

pretrained_D_dict = torch.load('/content/gdrive/Shareddrives/cici 544/style-transformer/2300_D.pth')

model_D.load_state_dict(pretrained_D_dict)
model_D.to(config.device)
print(config.discriminator_method)

# predict(config, vocab, model_D, test_iters)

vectors torch.Size([21760, 200])
load embedding took 0.82 s.
Vocab size: 21760
embed loaded.
Cond


In [27]:
from sklearn.metrics import precision_recall_fscore_support
predict(config, vocab, model_D, test_iters)

(0.915, 0.9150813559457653, 0.9149958347959051, None)
